In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from utils import run_ML

In [3]:
# !pip install tldextract

In [4]:
data_dir = "data/PhiUSIIL_Phishing_URL_Dataset.csv"
df = pd.read_csv(data_dir)

In [5]:
df.head(2)

,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,URLCharProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,https://www.southbankmosaics.com,31,www.southbankmosaics.com,24,0,com,100.0,1.000000,0.522907,0.061933,...,0,0,1,34,20,28,119,0,124,1
1,https://www.uni-mainz.de,23,www.uni-mainz.de,16,0,de,100.0,0.666667,0.032650,0.050207,...,0,0,1,50,9,8,39,0,217,1


In [32]:
smalldata = df.sample(n = 20000, random_state=1)

In [33]:
if 'PhiUSIIL' in data_dir:
    urldata = np.array(smalldata[['URLLength', 'DomainLength', 'IsDomainIP', \
           'URLSimilarityIndex', 'CharContinuationRate', 'TLDLegitimateProb',\
           'URLCharProb', 'TLDLength', 'NoOfSubDomain', 'HasObfuscation',\
           'NoOfObfuscatedChar', 'ObfuscationRatio', 'NoOfLettersInURL',\
           'LetterRatioInURL', 'NoOfDegitsInURL', 'DegitRatioInURL',\
           'NoOfEqualsInURL', 'NoOfQMarkInURL', 'NoOfAmpersandInURL',\
           'NoOfOtherSpecialCharsInURL', 'SpacialCharRatioInURL', 'IsHTTPS']])
    labels = smalldata.iloc[:,-1].values

In [34]:
run_ML(urldata, labels, "PhiUSIILmock", "standard")

Run:  0 , fold:  0
Train freq:  [6844, 9156]
kNN, LightGBM, Run:  0 , fold:  1
Train freq:  [6880, 9120]
kNN, LightGBM, Run:  0 , fold:  2
Train freq:  [6819, 9181]
kNN, LightGBM, Run:  0 , fold:  3
Train freq:  [6802, 9198]
kNN, LightGBM, Run:  0 , fold:  4
Train freq:  [6883, 9117]
kNN, LightGBM, ['kNN', 'LightGBM']
[1. 1.]


In [35]:
# df.columns.values
# urldata.shape

### Feature extraction

In [36]:
from utils import extract_features

In [37]:
# Example usage:
url = "http://www.example.com/path/to/==file.html"
url_features = extract_features(url)
print(url_features)

{'domain': 'www.example.com', 'num_subdomains': 2, 'contains_ip': 0, 'path_length': 20, 'num_path_segments': 3, 'uses_https': 0, 'file_extension': 'html', 'count_special_characters': 11, 'count_non_alphanumeric_characters': 11, 'TLD': 'com', 'count_obfuscated_characters': 0, 'letter_ratio_in_url': 0.7380952380952381, 'digit_ratio_in_url': 0.0, 'count_equals_in_url': 2, 'NoOfAmpersandInURL': 0, 'CharContinuationRate': 0.11904761904761904, 'ratio_obfuscated_characters': 0.0, 'NoOfQMarkInURL': 0}


In [38]:
# print(url_features.keys())

In [39]:
phish_url = []
for link in list(smalldata.iloc[:,0]):
    url_features = extract_features(link)
    phish_url.append(list(url_features.values())[1:])

In [40]:
phish_url_df = pd.DataFrame(phish_url, columns = list(url_features.keys())[1:])

In [41]:
# phish_url_df.head(2)

In [42]:
phish_url_df.iloc[:,5] = pd.Categorical(phish_url_df.iloc[:,5]).codes
phish_url_df.iloc[:,8] = pd.Categorical(phish_url_df.iloc[:,8]).codes

In [43]:
phish_url_df.head(2)

,num_subdomains,contains_ip,path_length,num_path_segments,uses_https,file_extension,count_special_characters,count_non_alphanumeric_characters,TLD,count_obfuscated_characters,letter_ratio_in_url,digit_ratio_in_url,count_equals_in_url,NoOfAmpersandInURL,CharContinuationRate,ratio_obfuscated_characters,NoOfQMarkInURL
0,2,0,0,1,1,0,5,5,276,0,0.736842,0.0,0,0,0.210526,0.0,0
1,2,0,0,1,1,0,5,5,101,0,0.791667,0.0,0,0,0.208333,0.0,0


In [44]:
run_ML(phish_url_df, labels, "PhiUSIILmock", "manual")

Run:  0 , fold:  0
Train freq:  [6844, 9156]
kNN, LightGBM, Run:  0 , fold:  1
Train freq:  [6880, 9120]
kNN, LightGBM, Run:  0 , fold:  2
Train freq:  [6819, 9181]
kNN, LightGBM, Run:  0 , fold:  3
Train freq:  [6802, 9198]
kNN, LightGBM, Run:  0 , fold:  4
Train freq:  [6883, 9117]
kNN, LightGBM, ['kNN', 'LightGBM']
[0.97 1.  ]


In [45]:
urldata.shape, phish_url_df.shape

((20000, 22), (20000, 17))